<a href="https://colab.research.google.com/github/LeaCarop/California_Housing/blob/main/Ajustes_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AJUSTE DE HIPERPARÁMETROS

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
housing = fetch_california_housing()   # a diferencia del otro dataset, este es más sencillo, solo tiene variables numéricas

# conjunto de entrenamiento, validación y prueba
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state=42)

In [3]:
#  para escalar todas las características y las devuelva en un mismo rango
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [11]:
def build_model(capas=1, neuronas=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(capas):
        model.add(keras.layers.Dense(neuronas, activation="relu"))
    model.add(keras.layers.Dense(1))     # regresión univariante (solo una neurona de salida)
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [13]:
#CREAR KERAS REGRESSOR
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model) # usa los hiperparámetros de build_model
# Ahora lo puedo usar como un regresor sklearn normal

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


In [15]:
#ENTRENAMIENTO
keras_reg.fit(x_train, y_train, epochs=100,
              validation_data=(x_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 1.3133 - val_loss: 4.2454
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5892 - val_loss: 0.5354
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5063 - val_loss: 0.5953
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4687 - val_loss: 0.4643
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4486 - val_loss: 0.5578
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4360 - val_loss: 0.5992
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4274 - val_loss: 0.6430
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4221 - val_loss: 0.4653
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4170 - val_loss: 0.3870
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4123 - val_lo

In [17]:
#EVALUACIÓN
mse_test = keras_reg.score(x_test, y_test)
# resultado muy similiar a train

162/162 [==============================] - 0s 834us/step - loss: 0.3654


### ENTRENAR VARIAS VARIANTES A LA VEZ

In [24]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "capas": [0, 1, 2, 3],
    "neuronas": np.arange(1, 100),         
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)   # funciona como una GridSearchCV
rnd_search_cv.fit(x_train, y_train, epochs=100,
                  validation_data=(x_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 1.4859 - val_loss: 1.3865
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6270 - val_loss: 0.6134
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5309 - val_loss: 0.5015
Epoch 4/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4884 - val_loss: 0.4642
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4626 - val_loss: 0.4385
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4467 - val_loss: 0.4257
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4396 - val_loss: 0.4202
Epoch 8/100
242/242 [==============================] - 1s 3ms/step - loss: 0.4364 - val_loss: 0.4110
Epoch 9/100
242/242 [==============================] - 1s 3ms/step - loss: 0.4320 - val_loss: 0.4115
Epoch 10/100
242/242 [========

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-3.81975790e-01 -1.05595676e+06 -3.81235838e-01 -4.44974234e-01
 -4.09169426e-01 -3.06512892e-01 -4.46771195e-01 -6.22782966e-01
 -3.42216005e-01             nan]
  category=UserWarning,


363/363 [==============================] - 1s 2ms/step - loss: 0.8133 - val_loss: 0.5736
Epoch 2/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3817 - val_loss: 0.8991
Epoch 3/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3917 - val_loss: 2.1305
Epoch 4/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3617 - val_loss: 1.1901
Epoch 5/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3489 - val_loss: 1.0899
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3444 - val_loss: 0.3588
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3346 - val_loss: 0.3331
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3253 - val_loss: 0.3432
Epoch 9/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3202 - val_loss: 0.3185
Epoch 10/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3190 - val_loss: 0.3129
E

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f6cc425cc10>,
                   param_distributions={'capas': [0, 1, 2, 3],
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6cbf8ae150>,
                                        'neuronas': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   verbose=2)

In [25]:
rnd_search_cv.best_params_  # # saco los mejores hiperparámetros para mi modelo

{'capas': 3, 'learning_rate': 0.01921050137607719, 'neuronas': 38}

In [26]:
rnd_search_cv.best_score_

-0.30651289224624634

In [27]:
rnd_search_cv.best_estimator_

In [28]:
rnd_search_cv.score(x_test, y_test)

162/162 [==============================] - 0s 869us/step - loss: 0.2857


-0.2857230007648468

In [29]:
model = rnd_search_cv.best_estimator_.model
model

In [30]:
model.evaluate(x_test, y_test)

162/162 [==============================] - 0s 886us/step - loss: 0.2857


0.2857230007648468